In [14]:
from tkinter import *
from random import sample, randint
from carte import *
from dessin import *
import pygame

def nombre(mines,n,click):
    compteurMines = 0
    grid = [[0]*n for _ in range(n)]
    for (i,j) in mines:
        v = voisins(n,i,j)
        for (a,b) in v:
            if not((a,b) in mines):
                grid[a][b] += 1
                
    for (x,y) in mines:
        grid[x][y] = "X"
        compteurMines += 1
    
    return grid,compteurMines

def neutre():
    global listeImages
    listeImages = ajouterImage("neutre")
    
def moderne():
    global listeImages
    listeImages = ajouterImage("moderne")
    
def flower():
    global listeImages
    listeImages = ajouterImage("flower")


#Dessine tous les éléments du menu : boutons, slider, image
def drawMenu(cnv):
    global widgets
    niveau = StringVar()
    niveau.set('normal')
    widgets.append(niveau)
    
    mon_menu = Menu(root)
    root.config(menu=mon_menu)
    themes = Menu(mon_menu, tearoff=0)
    themes.add_command(label="Neutre", command=neutre)
    themes.add_command(label="Moderne", command=moderne)
    themes.add_command(label="Flower Power", command=flower)
    mon_menu.add_cascade(label="Thèmes", menu=themes)
    
    imgMenu = PhotoImage(file="démineur.gif")
    cnv.create_image(250,300,image=imgMenu)
    
    facile = Radiobutton(root, variable=niveau, text="Facile",value="facile", font="arial 10")
    facile.pack(side="left", expand=1)
    widgets.append(facile)

    normal = Radiobutton(root, variable=niveau, text="Normal",value="normal", font="arial 10")
    normal.pack(side="left", expand=1)
    widgets.append(normal)

    difficile = Radiobutton(root, variable=niveau, text="Difficile",value="difficile", font="arial 10")
    difficile.pack(side="left", expand=1)
    widgets.append(difficile)

    slider = Scale(root, orient = "horizontal", command=taillePlateau, from_=15, to=100)
    slider.pack()
    widgets.append(slider)

#Elle est appelée à chaque clic sur le plateau et permet de générer la grille et/ou de découvrir les cases
def startGame(event):
    global states, widgets, compteurMines, lst_mines
    if screen == "game":
        click = (event.y//cellsize,event.x//cellsize)
        if lst_mines==[]:
            states,compteurMines,lst_mines = createMap(n,click,p,lst_mines)
            reGenMap()
            timer()
        if bPerdu == False and bGagné == False :
            widgets[len(widgets)-1]["image"] = listeImages[11]
            revelle(n,states,click,game, randint(1,2))            

#Elle est apellée quand on clique le sur le bouton start du menu 
#Et créé tous les widgets pour jouer
def startMenu():
    global p, game, caseDecouverte, mapFlag, screen, MAX_X, MAX_Y, R, widgets
    niv = widgets[0].get()
    
    if len(listeImages) == 0:
        neutre()
    
    caseDecouverte = [[0]*n for i in range(n)]
    mapFlag = [[0]*n for _ in range(n)]
    
    MAX_X,MAX_Y = n*cellsize,n*cellsize+100
    R = MAX_X//n
    game = createCanvas(MAX_X,MAX_Y,root)
    game.pack()
    menu.destroy()
    
    game.bind("<Button-1>", startGame)
    game.bind("<ButtonRelease-1>", smiley)
    game.bind("<Button-3>", drapeau)
    
    if niv == "facile":
        p = 0.05
    elif niv == "normal":
        p = 0.15
    else:
        p = 0.2
        
    grille(n,game,listeImages,cellsize)
    
    for i in range(1,len(widgets)):
        widgets[i].destroy()
        
    btnAide = Button(root, image= listeImages[12], command=aide)
    btnAide.pack()
    widgets.append(btnAide)
        
    btnReset = Button(root, image=listeImages[9], command=reset)
    btnReset.pack()
    widgets.append(btnReset)    
    screen = "game"

#Elle est appelé quand le bouton aide du jeu est appuyé et affiche l'image d'aide
def startAide():
    global game
    cnvAide.destroy()
    for i in range(1,len(widgets)):
        widgets[i].destroy()
    
    MAX_X,MAX_Y = n*cellsize,n*cellsize+100
    game = createCanvas(MAX_X,MAX_Y,root)
    game.pack()
    
    game.bind("<Button-1>", startGame)
    game.bind("<ButtonRelease-1>", smiley)
    game.bind("<Button-3>", drapeau)
    
    grille(n,game,listeImages,cellsize)
    
    btnAide = Button(root, image= listeImages[12], command=aide)
    btnAide.pack()
    widgets.append(btnAide)
        
    btnReset = Button(root, image=listeImages[9], command=reset)
    btnReset.pack()
    widgets.append(btnReset)
    
    timer()
    reGenMap()
    
    screen = "game"

#Elle permet de révéler les cases ou les zones à partir du clic
def revelle(n,grid,click,cnv,r):
    if r == 1:
        soudBloc_1.play()
    else:
        soudBloc_2.play()
    nombreDeTours = int((0.2//p)+1)
    x,y = click[0],click[1]
    compteurDeDrapeau = 0
    mineVoisin = 0
    L = []
    global lst_caseDecouvertes,caseDecouverte
    if caseDecouverte[x][y] == 1 or lst_caseDecouvertes == []:
        v = voisins(n,y,x)
        for i,j in v:
            if mapFlag[i][j] == 1:
                compteurDeDrapeau += 1
            elif grid[j][i] == "X" and mapFlag[j][i] != 2 and lst_caseDecouvertes != []:
                mineVoisin = (j,i)
        if compteurDeDrapeau == grid[x][y] or lst_caseDecouvertes == []:
            if mineVoisin != 0:
                perdu(mineVoisin,cnv)
            
            #Découvre les cases en faisant k tours qui dépendent de la difficulté
            L = voisins(n,click[0],click[1])
            while len(L) != 0:
                cases = L
                for (i,j) in cases:
                    if grid[i][j] == 0:
                        grid[i][j] = 9
                        caseDecouverte[i][j] = 1
                        v = voisins(n,i,j)
                        for (a,b) in v:
                            if caseDecouverte[a][b] == 0:
                                L.append((a,b))
                                caseDecouverte[a][b] = 1
                                lst_caseDecouvertes.append((a,b))
                    else:
                        if caseDecouverte[i][j] == 0:
                            caseDecouverte[i][j] = 1
                    lst_caseDecouvertes.append((i,j))
                    L.remove((i,j))
    else:
        if grid[x][y] == "X":
            perdu(click,cnv)
        else:
            lst_caseDecouvertes.append((x,y))
            caseDecouverte[x][y] = 1

#Vérifie si le joueur à découvert une mine si oui affiche perdu
#Et active la variable bPerdu (boolean Perdu)
def perdu(click,cnv):
    if states[click[0]][click[1]] == "X":
        soudPerdu.play()
        global bPerdu
        bPerdu = True
        draw(states,cnv,"perdu",listeImages,cellsize)
        highScores(chrono)
        widgets[len(widgets)-1]["image"] = listeImages[10]
        
#Elle est appelé quand le joueur à découvert toutes les mines et mis un drapeau sur chacune
#Elle affiche gagné et active la variable bGagné (boolean Gagné)
def gagné(flag, states):
    soudGagne.play()
    for (i,j) in flag:
        if states[j][i] != "X":
            return
    global bGagné
    bGagné = True
    draw(states,game,"gagné",listeImages,cellsize)

#Cette fonction dessine tout le plateau de jeu entièrement à partir des vaiable globales
def reGenMap():
    if bPerdu == False and bGagné == False:
        game.delete("all")
        grille(n,game,listeImages,cellsize)
        #Dessine les drapeaux
        for i in range(len(mapFlag)):
            for j in range(len(mapFlag)):
                if mapFlag[i][j] == 1:
                    game.create_image(cellsize*(i+0.5),cellsize*(j+0.5),image=listeImages[6])
                elif mapFlag[i][j] == 2:
                    game.create_image(cellsize*(i+0.5),cellsize*(j+0.5),image=listeImages[8])
                    
        #Parcours la liste des cases découvertes et les dessines
        for (i,j) in lst_caseDecouvertes:
            if not((i,j) in lst_mines):
                if states[i][j] != 9:
                    game.create_image(cellsize*(j+0.5),cellsize*(i+0.5),image=listeImages[states[i][j]-1])
                else:
                    game.create_image(cellsize*(j+0.5),cellsize*(i+0.5),image=listeImages[14])
#                    game.create_rectangle((j*cellsize,i*cellsize), ((j+1)*cellsize,(i+1)*cellsize), fill="white", outline='black')
        taille = 8
        
        #Afficher le chronomètre dans un afficheur SSD         
        afficheur((MAX_X/8)-18,(MAX_Y/8)-11,taille,splitNumber(chrono)[2],game)
        afficheur((MAX_X/8)-12,(MAX_Y/8)-11,taille,splitNumber(chrono)[1],game)
        afficheur((MAX_X/8)-6,(MAX_Y/8)-11,taille,splitNumber(chrono)[0],game)

        #Afficher le compteur de mine dans un afficheur SSD
        afficheur(1,(MAX_Y/8)-11,taille,splitNumber(compteurMines-len(flag))[2],game)
        afficheur(7,(MAX_Y/8)-11,taille,splitNumber(compteurMines-len(flag))[1],game)
        afficheur(13,(MAX_Y/8)-11,taille,splitNumber(compteurMines-len(flag))[0],game)

        game.after(50,reGenMap)
             
#drapeau ajoute à la liste des drapeau les coordonées du clic droit
#Et met 1 dans la carte des drapeaux
def drapeau(event):
    if screen == "game":
        x, y = event.x//cellsize, event.y//cellsize
        global mapFlag
        if mapFlag[x][y] == 1:
            soudDrapeau2.play()
            mapFlag[x][y] = 2
            flag.remove((x,y))
        elif mapFlag[x][y] == 2:
            mapFlag[x][y] = 0
        elif caseDecouverte[y][x] == 0:
            soudDrapeau1.play()
            flag.append((x,y))
            mapFlag[x][y] = 1
    
        if compteurMines-len(flag) == 0:
            gagné(flag, states)

#Incrémente le chronomètre toutes les secondes
def timer():
    global chrono
    if screen == "game":
        chrono += 1
        game.after(1000,timer)

#Quand le joueur perd ou gagne on détermine son highscore et on l'écrit dans un fichier .txt
def highScores(chrono):
    fichier = open("./HighScores.txt", 'r')
    score=0
    print(fichier.readline())
    if fichier.readline() != '':
        score = int(fichier.readline())
    fichier.close()
    
    fichier = open("./HighScores.txt", "w")
    if score > chrono or score == 0:
        fichier.write(str(chrono))
    fichier.close()

#Elle est appelé par le slider du menu pour
#déterminer la taille du plateau
def taillePlateau(t):
    global n
    n = int(t)

#Elle est appelé quand on appuie sur le bouton JOUER du menu aide
#Remet en fonctionnement le jeu, laisser en pause
def aide():
    global cnvAide
    screen="aide"
    for i in range(1,len(widgets)):
        widgets[i].destroy()
    w,h = 500,700
    game.destroy()
    cnvAide = createCanvas(w,h,root)
    cnvAide.pack()
    cnvAide.create_image(w/2, h/2, image=listeImages[13])
    btnReturn = Button(root, text="Jouer", command=startAide)
    btnReturn.pack()
    widgets.append(btnReturn)

#Réinitialise toutes les variables globales pour recommencer
def reset():
    global menu,screen,flag,lst_caseDecouvertes,lst_mines,states,bPerdu
    for i in range(1,len(widgets)):
        widgets[i].destroy()
    screen = "menu"
    del states[0:len(states)]
    del flag[0:len(flag)]
    del lst_caseDecouvertes[0:len(lst_caseDecouvertes)]
    del lst_mines[0:len(lst_mines)]
    del widgets[0:len(widgets)]
    bPerdu = False
    game.destroy()
    menu = createCanvas(500,600,root)
    drawMenu(menu)
    btnStart=Button(root, text="Start", command=startMenu)
    btnStart.pack()
    widgets.append(btnStart)
    
#Vérifie que le joueur n'a pas perdu pour réafficher l'image du sourire sur le bouton
def smiley(event):
    if widgets[len(widgets)-1]["image"] != "" and bPerdu == False:
        widgets[len(widgets)-1]["image"] = listeImages[9]

#Ajoute toutes les images dans une liste qui une fois retournée sera globale
def ajouterImage(theme):
    listeImages = []
    for i in range(1,7):
        listeImages.append(PhotoImage(file=theme+"/"+str(i)+".gif"))
    imgDrapeau = PhotoImage(file=theme+"/drapeau.gif"),
    imgMine = PhotoImage(file=theme+"/mine.gif"),
    imgInterrogation = PhotoImage(file=theme+"/interrogation.gif"),
    imgSourire = PhotoImage(file=theme+"/sourire.gif"),
    imgMort = PhotoImage(file=theme+"/mort.gif"),
    imgClic = PhotoImage(file=theme+"/click.gif"),
    imgAide = PhotoImage(file="aide.gif"),
    imgMenuAide = PhotoImage(file="menuAide.gif")
    listeImages.append(imgDrapeau)
    listeImages.append(imgMine)
    listeImages.append(imgInterrogation)
    listeImages.append(imgSourire)
    listeImages.append(imgMort)
    listeImages.append(imgClic)
    listeImages.append(imgAide)
    listeImages.append(imgMenuAide)
    listeImages.append(PhotoImage(file=theme+"/0.gif"))
    listeImages.append(PhotoImage(file=theme+"/fond.gif"))
    return listeImages

#Toutes les variable globales du jeu
states = []
flag = []
widgets = []
lst_caseDecouvertes = []
lst_mines = []

bPerdu = False
bGagné = False
premierClick = True

cellsize = 25
n = 15
MAX_X,MAX_Y = 0,0
R = 0
chrono = 0

screen = "menu"
global root
root=Tk()

pygame.mixer.pre_init(44100, -16, 1, 512)
pygame.mixer.init()

menu = createCanvas(500,600,root)
menu.pack()

listeImages = []

soudDrapeau1 = pygame.mixer.Sound("musiques/drapeau1.wav")
soudDrapeau2 = pygame.mixer.Sound("musiques/drapeau2.wav")
soudGagne = pygame.mixer.Sound("musiques/gagne.wav")
soudPerdu = pygame.mixer.Sound("musiques/perdu.wav")
soudBloc_1 = pygame.mixer.Sound("musiques/bloc_1.wav")
soudBloc_2 = pygame.mixer.Sound("musiques/bloc_2.wav")

imgMenu = PhotoImage(file="démineur.gif")
drawMenu(menu)
btnStart=Button(root, text="Start", command=startMenu)
btnStart.pack()
widgets.append(btnStart)

root.mainloop()

